<개요>
1. 데이터 로드
2. 레시피 s

### Data Loading


- 오라클에서 가져오는 방법 추가

In [37]:
import pandas as pd
data = pd.read_csv(r'C:\Users\HwijunKwon\Downloads\RecipeData.csv')

### Doc2Vec으로 임베딩하기

- 전처리 없이 샘플 10000개 임베딩
- 1. recipe_step을 형태소 분석기로 전처리
- 2. gensim doc2vec으로 유사도 분석

In [38]:
import random 
import numpy as np

np.random.seed(77)
toy = data[["recipe_title", "recipe_step"]].sample(1000) # toy 데이터 만들기 표본 1000개

In [27]:
toy #레시피 타이틀, 레시피 스텝으로 구성

,recipe_title,recipe_step
25036,떡잡채- 수미네반찬 10회,"['#1 소고기(150g)은 양조간장(1큰술), 설탕(1작은술), 마늘(1큰술), ..."
26159,오징어볶음과 오징어덮밥으로도 간단하게~,['#1 오징어를 지난번 다리를 다 먹어버렸어요.그래서 오늘은 이렇게 몸통만 2마리...
20386,순정 느타리버섯으로 만드는 버섯 겉절이,"['#1 순정 느타리버섯은 적당한 크기로 손으로 찢어 준다.', '#2 찢은 버섯은..."
113414,[크림 파스타] 정통 카르보나라(까르보나라),"['#1 그릇에 계란과 계란 노른자, 치킨 스톡, 페코리노 로마노, 후추를 넣고 섞..."
21341,새우계란찜,['#1 새우는 머리를 떼고 몸통 부분의 껍질을 벗겨줍니다. 그리고 새우 등쪽과 배...
...,...,...
94272,굴소스 어묵볶음,"['#1 파프리카는 먹기 좋은 크기로 썰어주세요.', '#2 달군 팬에 식용유 두르..."
168638,겨울을 더 건강하게! 겨울제철 음식 최고의 궁합♡,[]
143248,달콤한 애플 군만두,"['#1 사과는 깨끗이 씻어 껍질채 잘게 썰어 준비해주세요~', '#2 작은 냄비나..."
43233,시래기된장국,"['#1 엄마가 삶아주신 무청시래기입니다', '#2 집에가져와서 작에 썰어줘요반은 ..."


In [34]:
from konlpy.tag import Kkma, Komoran, Okt, Hannanum #Mecab은 윈도우에서 작동 불가능
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

okt = Okt()
# kkma = Kkma()
# komoran = Komoran()
# hannanum = Hannanum()

In [35]:
import re
tagged_corpus_list = []
cleaned_tagged_corpus_list = []

for index, row in tqdm(toy.iterrows(), total=len(toy)):
  cleaned_text = re.sub(r'[^A-Za-z0-9가-힣\s]+', '', row['recipe_step'])
  words = okt.morphs(cleaned_text)
  tag = row['recipe_title']
  tagged_corpus_list.append(TaggedDocument(words=words, tags=[tag]))


100%|██████████| 1000/1000 [00:20<00:00, 49.59it/s]


In [41]:
tagged_corpus_list[10]

TaggedDocument(words=['1', '재료', '를', '준비', '해주세요', '2', '시금치', '두', '단', '씻어서', '데칠', '준비', '해', '뒀네요', '시금치', '는', '다듬어서', '깨끗하게', '씻어서', '데', '쳐주세요', '3', '끓는', '물', '에', '소금', '넣고', '데', '쳐주면', '시금치', '가', '파릇파릇하게', '데', '쳐진답니다', '데친', '다음', '찬물', '에', '헹궈', '서', '소금', '참기름', '통깨', '마늘', '넣고', '무쳐주면', '되네요', '4', '평소', '에는', '계란', '지단', '얇게', '해서', '채', '썰어서', '넣어', '줬는데', '오늘', '은', '두', '툼', '하', '게', '부쳐', '봤어요', '5', '당근', '도', '채', '썰어서', '소금', '간', '해서', '볶아', '줬어요', '6', '햄', '도', '살짝', '볶아주고', '7', '우엉', '이', '있으면', '좋은데', '없어서', '대신', '어묵', '볶음', '넣어', '줬어요', '어묵', '볶다가', '간장', '물엿', '2', '개', '씩', '넣고', '볶아', '줬어요', '8', '김밥', '재료', '준비', '끝', '9', '밥', '에', '소금', '참기름', '통깨', '로', '간', '해줬고요', '10', '김위', '에', '밥', '얇게', '깔고', '재료', '순서대로', '쭉', '올려서', '돌돌', '말아주면', '되네요', '김발', '사용', '해도', '좋고', '저', '는', '그냥', '마는', '게', '편해서', '김발', '없이', '말', '아', '줬어요', '11', '참기름', '발라', '서', '잘라주기만', '하면', '되죠', '12', '언제', '먹어도', '질리', '지', '않는', '김밥', '완성', '김밥', '이랑', '잘', '어울리

In [43]:
from gensim.models import doc2vec

model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# Vocabulary 빌드
model.build_vocab(tagged_corpus_list)

# 수정된 부분: doctags 대신 key_to_index 사용
print(f"Tag Size: {len(model.dv.key_to_index)}", end=' / ')

# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=10)

# 모델 저장
model.save('recipe.doc2vec')


Tag Size: 998 / 

In [46]:
similar_doc = model.dv.most_similar('김밥 맛있게 싸는법~')
print(similar_doc)

[('가지볶음♡', 0.7382035851478577), ('아삭 건강한 깍두기 담그는법~', 0.7284537553787231), ('꼬마김밥', 0.7279257774353027), ('고구마 줄기 김치 담그는 법', 0.7142994999885559), ('<오이파프리카냉국> 남은파프리카 활용법~ 5분만에 뚝딱 완성', 0.7117801308631897), ('흔한 물김치가 아닌 겉절이로 맛있게 돌나물오이김치', 0.7085570693016052), ('오이소박이 만드는법~~다오네장아찌원액으로 무르지않는 오이소박이 만드는법', 0.7077771425247192), ('꽁치 통조림으로 만든 꽁치 김치찜..', 0.7041850090026855), ('새콤달콤 초간단 비빔만두 양념장', 0.6960198879241943), ('배추손질법 제대로하기!', 0.6959057450294495)]


In [51]:
similar_doc_2 = model.dv.most_similar('달콤한 애플 군만두')
print(similar_doc_2)

[('한그릇요리 뚝배기 치즈밥 만들기', 0.8521183133125305), ('명절에 남은 모듬전으로 전 볶음 만들어 보세요.', 0.8457191586494446), ('삼치조림 30분완성,직장맘', 0.8448374271392822), ('트리플 치즈샌드', 0.844165563583374), ('초간단 간식 어묵꼬지전~*', 0.8380562663078308), ('쉽게 만드는 떡볶이', 0.8367215991020203), ('매운 까르보나라 떡볶이 만들기♥', 0.8285486102104187), ('해투야간매점 지성쌈밥 만들기', 0.8285279870033264), ('버터갈릭옥수수 이름만들어도 침나오네 ^^', 0.8284329771995544), ('<출출할때 고소한 영양간식, 시리얼바>', 0.8216112852096558)]


### 해결해야할 과제


1. 레시피에서 하는 요리에 대한 정의가 필요함
  - 레시피 타이틀 전처리
  - 레시피 태그 전처리 
2. 레시피에 따라서 2인분, 3인분, 4인분 다양함

In [67]:
data["recipe_summary1"].value_counts()

2인분       57219
4인분       31408
1인분       25937
3인분       19687
6인분 이상    16377
5인분        3670
Name: recipe_summary1, dtype: int64